<a href="https://colab.research.google.com/github/Rajaguhan437/Cell_Detect-Quantitative-Representation/blob/main/SAM_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd "/content/drive/MyDrive/Ninti/"

!git clone https://github.com/facebookresearch/segment-anything.git
!cd segment-anything; pip install -e .

In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
from segment_anything import sam_model_registry,  SamAutomaticMaskGenerator, SamPredictor

In [ ]:
if not os.path.exists("/content/drive/MyDrive/Ninti/segment-anything/sam_vit_h_4b8939.pth"):
  !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -O /content/drive/MyDrive/Ninti/segment-anything/sam_vit_h_4b8939.pth

In [ ]:
### initialize the SAM

sam_chkpt = "/content/drive/MyDrive/Ninti/segment-anything/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_chkpt)
sam.to(device=device)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.99, #a filtering threshold in [0,1], using the model's predicted mask quality
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100  #requires open-cv to run post-processsing
)

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Ninti/unzipped_dataset/images/image-1.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
masks = mask_generator.generate(image)

### Printing no. of masks and keys attached to each of them
print("Number of masks generated:", len(masks))
print(masks[0].keys())

In [ ]:
import pycocotools
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    mask_num = 1  # initialize mask number
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
        # Get segmentation center and display mask number
        y, x = np.where(m)
        center = np.array([int(np.mean(x)), int(np.mean(y))])
        ax.text(center[0], center[1], str(mask_num), color='white', fontsize=10, ha='center', va='center')
        mask_num += 1  # increment mask number

    ax.imshow(img)

### Displaying the masks, witha number identfying each of them
plt.figure(figsize=(5.12, 5.12))
plt.imshow(image)
show_anns(masks[:]) ## use masks[:-x] to only display the masks of the list
plt.axis('off')

### Saving this image
plt.savefig('/content/drive/MyDrive/Ninti/segment_overlay.jpg', dpi=300, bbox_inches='tight')
plt.show()